Importing

In [2]:
import pandas as pd
import numpy as np
import pandas as pd
from sklearn import preprocessing, model_selection, neighbors
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, precision_score
import matplotlib.pyplot as plt
import seaborn as sns
import os
!pip install imbalanced-learn
from imblearn.over_sampling import SMOTE

##Preprocessing

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
import seaborn as sns
import matplotlib.pyplot as plt

# 1. Load and Preprocess the Dataset
df = pd.read_csv("creditcard.csv")
df.fillna(df.mean(), inplace=True)  # Handle missing values
df.drop_duplicates(inplace=True)   # Remove duplicates
df['Class'] = df['Class'].astype(int)  # Ensure 'Class' is integer

# 2. Reduce Dataset to 994 Rows
rows_needed = 994
test_size = (len(df) - rows_needed) / len(df)
df, _ = train_test_split(df, test_size=test_size, random_state=42)

# Save the reduced dataset
df.to_csv('reduced_dataset.csv', index=False)

# 4. Prepare for SMOTE
X = df.drop('Class', axis=1)
y = df['Class']

# Standardize the Features (optional but recommended for SMOTE)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Apply SMOTE
from imblearn.over_sampling import SMOTE

# Get the minority class sample count
minority_class_count = y.value_counts().min()

# Ensure k_neighbors is <= minority_class_count - 1
k_neighbors = min(3, minority_class_count - 1)  # Use 3 if possible, otherwise adapt

# Apply SMOTE
smote = SMOTE(random_state=42, k_neighbors=k_neighbors)
X_resampled, y_resampled = smote.fit_resample(X_scaled, y)

# 5. Check Resampled Data Distribution
print("Class distribution after SMOTE:")
print(pd.Series(y_resampled).value_counts())

# If needed, save the resampled dataset
resampled_df = pd.DataFrame(X_resampled, columns=X.columns)
resampled_df['Class'] = y_resampled

# Filter to get exactly 994 samples per class
class_0 = resampled_df[resampled_df['Class'] == 0]
class_1 = resampled_df[resampled_df['Class'] == 1]

# Sample with replacement if necessary to ensure 994 samples per class
class_0_sampled = class_0.sample(n=5000, random_state=42, replace=True)
class_1_sampled = class_1.sample(n=5000, random_state=42, replace=True)

# Combine the two classes and shuffle
final_resampled_df = pd.concat([class_0_sampled, class_1_sampled]).sample(frac=1, random_state=42)

# Save the resampled dataset
final_resampled_df.to_csv('resampled_dataset.csv', index=False)

# Check the final distribution
print("Class distribution after SMOTE and filtering:")
print(final_resampled_df['Class'].value_counts())

Class distribution after SMOTE:
Class
0    991
1    991
Name: count, dtype: int64
Class distribution after SMOTE and filtering:
Class
1    5000
0    5000
Name: count, dtype: int64


##DecisionTree

In [ ]:
df=final_resampled_df
df

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
1384,-1.961320,0.167713,2.174528,-4.619108,4.429132,0.597235,-2.309251,-1.001462,0.083107,-2.515054,...,0.597593,-0.793169,-0.713279,0.156200,3.000270,0.893196,1.566930,1.774738,-0.459750,1
169,0.393891,-0.065345,-0.097758,1.231041,0.363636,-0.001019,0.486440,0.004906,0.028865,0.776754,...,0.320877,1.872824,-0.046152,0.351824,-1.286661,1.266896,-0.024429,-0.207955,-0.222869,0
946,1.145960,0.767093,0.129589,0.172755,0.574301,-0.197210,-0.431581,-0.042034,-0.161734,0.191762,...,-0.032362,0.043342,0.118560,0.651015,0.522580,-1.157256,0.043452,0.111391,-0.453858,0
868,0.351867,-0.077379,0.357765,-0.083228,-0.669529,0.981850,0.452891,0.579209,0.208803,-0.370734,...,0.193678,0.394169,-0.498462,-3.107090,0.001818,-1.040695,0.166939,0.354103,-0.420582,0
756,0.990183,-2.720145,-1.149441,-0.510612,-0.465290,-0.883196,0.973297,1.050939,-0.149358,1.669201,...,-1.306490,0.377576,0.110237,-0.233921,-0.645969,1.181689,-5.526867,2.875812,1.164915,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1403,-1.244274,-6.894278,5.299967,-12.683483,4.169727,-6.166681,-2.851468,-8.044200,8.715356,-3.341916,...,2.193172,-1.673437,-0.963281,0.174907,2.735994,-0.214641,3.114413,1.715202,-0.094362,1
1451,-1.418824,-5.328767,4.611923,-10.913952,4.239430,-4.667610,-2.739526,-6.485958,6.802939,-3.159058,...,1.840003,-1.478497,-0.909711,0.178418,2.807668,0.035210,2.776487,1.732468,-0.175286,1
1410,-1.980720,0.172943,2.178030,-4.635145,4.444083,0.601806,-2.318840,-0.999878,0.078155,-2.514976,...,0.597066,-0.792758,-0.715333,0.165452,3.016303,0.899177,1.571981,1.779703,-0.459928,1
69,1.091673,-0.583504,-0.263899,0.997676,-0.218150,0.663218,-0.741591,-0.030517,0.178706,-0.070170,...,0.201634,0.200785,-0.181670,0.954399,0.963244,0.549472,-0.197003,0.198891,-0.431318,0


In [ ]:
X = df.drop('Class', axis=1)
y = df['Class']  # Re-assign y after type conversion

In [ ]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
y_train

,Class
471,0
878,0
1262,1
73,0
1048,1
...,...
1452,1
1722,1
1926,1
1121,1


In [ ]:
dt = DecisionTreeClassifier(random_state = 5)
dt.fit(X_train, y_train)
y_pred_DT = dt.predict(X_test)
cm = confusion_matrix(y_test, y_pred_DT)
cm

array([[1013,    0],
       [   0,  987]])

In [ ]:
precision_score(y_test, y_pred_DT)


1.0

In [ ]:
accuracy = dt.score(X_test, y_test)
print("DT Accuracy:",accuracy)
print(classification_report(y_test, y_pred_DT))

DT Accuracy: 1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1013
           1       1.00      1.00      1.00       987

    accuracy                           1.00      2000
   macro avg       1.00      1.00      1.00      2000
weighted avg       1.00      1.00      1.00      2000



In [ ]:
import pandas as pd
y_pred = y_pred_DT[:994]  # Keep only the first 994 predictions

submission = pd.DataFrame({
    'ID': range(len(y_pred)),
    'Prediction': y_pred
})

submission.to_csv('submission_DT.csv', index=False)


##RandomForest

In [ ]:
rf = RandomForestClassifier(random_state = 5)
rf.fit(X_train, y_train)
y_pred_RF = rf.predict(X_test)

In [ ]:
cm = confusion_matrix(y_test, y_pred_RF)
cm

array([[1013,    0],
       [   0,  987]])

In [ ]:
precision_score(y_test, y_pred_RF)

1.0

In [ ]:
accuracy = rf.score(X_test, y_test)
print("RF Accuracy:",accuracy)
print(classification_report(y_test, y_pred_RF))

RF Accuracy: 1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1013
           1       1.00      1.00      1.00       987

    accuracy                           1.00      2000
   macro avg       1.00      1.00      1.00      2000
weighted avg       1.00      1.00      1.00      2000



In [ ]:
import pandas as pd
y_pred = y_pred_RF[:994]  # Keep only the first 994 predictions

submission = pd.DataFrame({
    'ID': range(len(y_pred)),
    'Prediction': y_pred
})

submission.to_csv('submission_RF.csv', index=False)


##NN

In [ ]:
from sklearn.metrics import classification_report, accuracy_score
from sklearn.neural_network import MLPClassifier


In [ ]:
mlp = MLPClassifier(hidden_layer_sizes=(100, 50), activation='relu', solver='adam', max_iter=500, random_state=42)

mlp.fit(X_train, y_train)



MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=500, random_state=42)

In [ ]:
y_pred_NN = mlp.predict(X_test)



In [ ]:
accuracy = accuracy_score(y_test, y_pred_NN)
print(f"Accuracy: {accuracy:.2f}")
print("Classification Report:")
print(classification_report(y_test, y_pred_NN))


Accuracy: 1.00
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1013
           1       1.00      1.00      1.00       987

    accuracy                           1.00      2000
   macro avg       1.00      1.00      1.00      2000
weighted avg       1.00      1.00      1.00      2000



In [ ]:
import pandas as pd
y_pred = y_pred_NN[:994]  # Keep only the first 994 predictions

submission = pd.DataFrame({
    'ID': range(len(y_pred)),
    'Prediction': y_pred
})

submission.to_csv('submission_NN.csv', index=False)


##Voting

In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import mean_squared_error
# Create a Voting Classifier
voting_reg = VotingClassifier(estimators=[
    ('DT', dt), ('rf', rf), ('mlp', mlp)
])

# Train and evaluate the voting regressor
voting_reg.fit(X_train, y_train)
y_pred_Voting = voting_reg.predict(X_test)

# Use y_pred_Voting instead of y_pred to calculate MSE
mse_voting = mean_squared_error(y_test, y_pred_Voting)
print(f"Voting Regressor MSE: {mse_voting:.4f}")
print(f"Voting Regressor MSE: {mse_voting:.4f}")


Voting Regressor MSE: 0.0000
Voting Regressor MSE: 0.0000


In [ ]:
import pandas as pd
y_pred = y_pred_Voting[:994]  # Keep only the first 994 predictions

submission = pd.DataFrame({
    'ID': range(len(y_pred)),
    'Prediction': y_pred
})

submission.to_csv('submission_Voting.csv', index=False)
